<a href="https://colab.research.google.com/github/DomPTech/Langchain-Agents-Demo/blob/main/langchain_agents_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
  langchain==0.3.25 \
  langchain-openai==0.3.22 \
  langchain-experimental==0.3.4 \
  numexpr==2.11.0 \
  google-search-results==2.4.2 \
  wikipedia==1.4.0 \
  sqlalchemy==2.0.41

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


To run this notebook, you need to use an OpenAI LLM. Here we setup the LLM used for the whole project, with the api key being stored as a Colab Secret.

In [ ]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
    or userdata.get('APIKey')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI

# Initialize the model
llm = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-4.1-mini",
    temperature=0.0,
)

In [ ]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [ ]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

# Define the table
temperatures = Table(
    "temperatures",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("city", String(20), nullable=False),
    Column("temperature_c", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [ ]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [ ]:
from datetime import datetime

# Example observations
observations = [
    [1, 'Springfield', 5.0, datetime(2023, 1, 1)],
    [2, 'Springfield', 6.5, datetime(2023, 1, 2)],
    [3, 'Springfield', 4.0, datetime(2023, 1, 3)],
    [4, 'Springfield', 3.5, datetime(2023, 1, 4)],
    [5, 'Springfield', 2.0, datetime(2023, 1, 5)],
    [6, 'Shelbyville', -1.0, datetime(2023, 1, 1)],
    [7, 'Shelbyville', -0.5, datetime(2023, 1, 2)],
    [8, 'Shelbyville', -2.0, datetime(2023, 1, 3)],
    [9, 'Shelbyville', -3.0, datetime(2023, 1, 4)],
    [10, 'Shelbyville', -4.0, datetime(2023, 1, 5)],
]

In [ ]:
from sqlalchemy import insert

# Function to insert observations
def insert_obs(obs):
    stmt = insert(temperatures).values(
        obs_id=obs[0],
        city=obs[1],
        temperature_c=obs[2],
        date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [ ]:
for obs in observations:
    insert_obs(obs)

In [ ]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain

db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)

### Agent type #1: Zero Shot React

The **Zero-Shot Agent** gets its name because it can attempt a task immediately without needing prior examples or multiple interactions.

- **Zero-shot** → The agent sees the input **once** and produces an answer right away. It doesn’t learn from previous steps or examples.
- **No memory** → Unlike a conversational agent, it **cannot recall previous questions or answers**. Each interaction is independent.

Basically, it's like asking someone a question without giving any context or hints and expecting them to answer correctly the first time.

This method uses a *toolkit* (a set of related tools designed to be used together) instead of a custom array of `tools`. For this use case, we will use `SQLDatabaseToolkit`.

**Important Note:** *When interacting with agents, it is incredibly important to set the `max_iterations` parameters because agents can get stuck in infinite loops, which will consume your API tokens (and either exceed your quota or cost you more moeny). The default value is 15 to allow for many tools and complex reasoning, but for most applications you should keep it much lower.*

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=5
)

Let's test our newly created agent by asking it a question that involves a math operation over the temperatures.

In [ ]:
result = agent_executor.invoke(
    "What is the difference in temperature between Springfield and Shelbyville on January 3rd, "
    "multiplied by the difference on January 4th?"
)
print(f"Result: {result['output']}")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: temperaturesThought: There is only one table named "temperatures." I should check the schema of this table to understand what columns it contains and how to query temperature data for Springfield and Shelbyville on January 3rd and 4th.
Action: sql_db_schema
Action Input: temperatures
CREATE TABLE temperatures (
	obs_id INTEGER NOT NULL, 
	city VARCHAR(20) NOT NULL, 
	temperature_c FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from temperatures table:
obs_id	city	temperature_c	date
1	Springfield	5.0	2023-01-01
2	Springfield	6.5	2023-01-02
3	Springfield	4.0	2023-01-03
*/Thought: I need to get the temperatures for Springfield and Shelbyville on January 3rd and January 4th, then calculate the difference in temperature between the two cities on each day, and finally multiply those two differences.

Action: sql_db_query_checker
Action Input: 
SELECT city, temperature_c, date FROM

### Agent type #2: Conversational React

If we want an AI assistant that can remember what we’ve talked about and reason about tasks, we can use a **Conversational ReAct Agent** in LangChain.

- **Conversational** → The agent keeps track of the **chat history**, allowing it to respond in context and remember details from earlier in the conversation.

- **ReAct** (short for **Reason + Act**) → The agent doesn’t just answer directly. Instead, it:
  1. **Reasons** about the problem step by step.
  2. **Decides** if it needs to use a tool (like a database, calculator, or search engine).
  3. **Acts** by calling the tool.
  4. **Continues reasoning** with the result to produce the final answer.

This combination makes the agent feel like a smart assistant that can chat naturally and performs tasks autonomously.


We will use the math tool in this example and load it as below:

In [ ]:
# Use the modern tool-based approach instead of deprecated LLMMathChain
from langchain_core.tools import tool
import math
import numexpr

@tool
def calculator(expression: str) -> str:
    """Calculate expression using Python's numexpr library.

    Expression should be a single line mathematical expression
    that solves the problem.

    Examples:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
        "10000 * (1 + 0.08)**5" for compound interest
    """
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

tools = [calculator]

In [ ]:
from langchain.memory import ConversationBufferMemory

# The memory type being used here is a simple buffer memory to allow us to remember previous steps in the reasoning chain.
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

/tmp/ipython-input-2595623399.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


The following examples rely on a **scratchpad**, which is like a working memory for the agent while it is reasoning through a problem. It’s a place where the agent can keep track of its intermediate thoughts, calculations, and tool usage before giving a final answer.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.base import RunnableSerializable
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.tools import tool

# Add a final_answer tool
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

# Add tools
tools = [final_answer, calculator]

# First, create a prompt that forces the LLM to analyze the history
history_analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Analyze the conversation history below and identify any calculations that have already been performed. "
        "Extract the results of these calculations so they can be reused instead of recalculating."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "What calculations have been done and what are their results?"),
])

# Then the main agent prompt
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user. "
        "IMPORTANT: Use the analysis of previous calculations to avoid recalculating."
    )),
    ("human", "Previous calculations analysis: {history_analysis}"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# define the agent runnable with history analysis first
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | {
        "history_analysis": lambda x: llm.invoke(history_analysis_prompt.format_messages(
            chat_history=x["chat_history"]
        )).content,
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | agent_prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

# create tool name to function mapping as per guide
name2tool = {tool.name: tool.func for tool in tools}

class CustomAgentExecutor:
    def __init__(self, max_iterations: int = 5):
        self.max_iterations = max_iterations
        self.chat_history = []  # Simple list to store conversation history
        self.agent = agent

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []

        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })

            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)

            # Handle ALL tool calls, not just the first one
            if tool_call.tool_calls:
                for tool_call_obj in tool_call.tool_calls:
                    tool_name = tool_call_obj["name"]
                    tool_args = tool_call_obj["args"]
                    tool_call_id = tool_call_obj["id"]

                    # execute the tool
                    tool_out = name2tool[tool_name](**tool_args)

                    # add the tool output to the agent scratchpad
                    tool_exec = ToolMessage(
                        content=f"{tool_out}",
                        tool_call_id=tool_call_id
                    )
                    agent_scratchpad.append(tool_exec)

                    # add a print so we can see intermediate steps
                    print(f"{count}: {tool_name}({tool_args}) -> {tool_out}")

                count += 1

                # Check if any tool call is the final answer tool
                if any(tc["name"] == "final_answer" for tc in tool_call.tool_calls):
                    # Get the final answer from the final_answer tool
                    final_tool_call = next(tc for tc in tool_call.tool_calls if tc["name"] == "final_answer")
                    final_answer = final_tool_call["args"]["answer"]
                    break
            else:
                # no tool call, we have a final answer
                final_answer = tool_call.content
                break

        # Add to conversation history ONLY the human input and final AI response
        # This preserves memory without corrupting it with tool calls
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=final_answer)
        ])

        # return the final answer in dict form
        return {"output": final_answer}

# Initialize the custom agent executor
conversational_agent = CustomAgentExecutor()

In [ ]:
# First question
result = conversational_agent.invoke("What is 10000 * (1 + 0.08)**5?")
print(f"Result: {result['output']}")

0: calculator({'expression': '10000 * (1 + 0.08)**5'}) -> 14693.280768000006
1: final_answer({'answer': 'The value of 10000 * (1 + 0.08)^5 is approximately 14693.28.', 'tools_used': ['functions.calculator']}) -> {'answer': 'The value of 10000 * (1 + 0.08)^5 is approximately 14693.28.', 'tools_used': ['functions.calculator']}
Result: The value of 10000 * (1 + 0.08)^5 is approximately 14693.28.


Let's see what happens if we try to answer the question that is related to the previous one:

In [ ]:
result = conversational_agent.invoke(
    "If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)
print(f"Result: {result['output']}")

0: calculator({'expression': '15000 * (1 + 0.08)**5 - 14693.28'}) -> 7346.641152000009
1: final_answer({'answer': 'If we start with $15,000 and follow the same 8% annual growth for 5 years with compound interest, we would have approximately $7,346.64 more compared to the previous scenario where we started with $10,000.', 'tools_used': ['functions.calculator']}) -> {'answer': 'If we start with $15,000 and follow the same 8% annual growth for 5 years with compound interest, we would have approximately $7,346.64 more compared to the previous scenario where we started with $10,000.', 'tools_used': ['functions.calculator']}
Result: If we start with $15,000 and follow the same 8% annual growth for 5 years with compound interest, we would have approximately $7,346.64 more compared to the previous scenario where we started with $10,000.


### Agent type #3: React Docstore

A **ReAct Docstore Agent** is an AI assistant that can search through documents and reason about the information it finds to answer questions. It only has two functions: "Search" and "Lookup."

A **Docstore** is a database or collection of documents that the agent can query to find information.

With "Search" it will bring up a relevant article and with "Lookup" the agent will find the right piece of information in the article.

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool

@tool
def Search(query: str) -> str:
    """Search Wikipedia for information about a topic."""
    try:
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)
    except Exception as e:
        return f"Error searching Wikipedia: {e}"

@tool
def Lookup(term: str) -> str:
    """Look up a specific term or phrase in Wikipedia."""
    try:
        wiki = WikipediaAPIWrapper()
        return wiki.run(term)
    except Exception as e:
        return f"Error looking up term: {e}"

tools = [Search, Lookup]

In [ ]:
# Create a custom agent executor for docstore tools
docstore_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant that can search and lookup information."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

docstore_agent_runnable = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | docstore_prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

docstore_name2tool = {tool.name: tool.func for tool in tools}

class DocstoreAgentExecutor:
    def __init__(self, max_iterations: int = 3):
        self.max_iterations = max_iterations
        self.agent = docstore_agent_runnable

    def invoke(self, input: str) -> dict:
        count = 0
        agent_scratchpad = []

        while count < self.max_iterations:
            tool_call = self.agent.invoke({
                "input": input,
                "agent_scratchpad": agent_scratchpad
            })

            agent_scratchpad.append(tool_call)

            if not tool_call.tool_calls:
                final_answer = tool_call.content
                break

            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = docstore_name2tool[tool_name](**tool_args)

            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)

            print(f"{count}: {tool_name}({tool_args}) = {tool_out[:100]}...")
            count += 1

        return {"input": input, "output": final_answer}

docstore_agent = DocstoreAgentExecutor()

We can ask it a question about when the last Apollo mission was.

In [ ]:
result = docstore_agent.invoke("When was the last Apollo mission?")
print(f"Result: {result['output']}")

0: Lookup({'term': 'Apollo program'}) = Page: Apollo program
Summary: The Apollo program, also known as Project Apollo, was the United State...
Result: The last Apollo mission was Apollo 17, which took place in December 1972. It was the final mission of the Apollo program that landed astronauts on the Moon.
